## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import cloudpickle



## CARGAR LOS DATOS

### Ruta del proyecto

In [2]:
directorio = r"C:\Users\DataScience\ONLINE_DS_THEBRIDGE_DaniCastillo\ML_LEAD SCORING\01_src\01_data_sample"

### Nombre del fichero de datos

In [3]:
# Cargar los datos originales
nombre_fichero_datos = 'Leads.csv'


In [4]:
ruta = os.path.join(directorio, nombre_fichero_datos)

### Cargar los datos

In [5]:
df = pd.read_csv(ruta, sep=";",index_col=0,)
df

,origen,fuente,no_enviar_email,no_llamar,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,ult_actividad,ambito,ocupacion,conociste_google,conociste_revista,conociste_periodico,conociste_youtube,conociste_facebook,conociste_referencias,score_actividad,score_perfil,descarga_lm
id,,,,,,,,,,,,,,,,,,,,
660737,API,Chat,No,No,0,0.0,0,0.00,Page Visited on Website,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660728,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,Business Administration,Student,No,No,No,No,No,No,14.0,20.0,Yes
660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,Media and Advertising,Unemployed,No,No,No,No,No,No,13.0,17.0,No
660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,Select,Unemployed,No,No,No,No,No,No,15.0,18.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,IT Projects Management,Unemployed,No,No,No,No,No,No,15.0,17.0,No
579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,Media and Advertising,Unemployed,No,No,No,No,No,No,14.0,19.0,Yes
579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,Business Administration,Unemployed,No,No,No,No,No,No,13.0,20.0,Yes


### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [6]:
ruta_variables = r"C:\Users\DataScience\PROYECTOS_ML\leadscoring\02_Datos\03_Trabajo"
nombre_variables_finales = os.path.join(ruta_variables, "variables_finales.pickle")

# Cargar el archivo
pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['ambito_IT Projects Management',
 'ambito_Select',
 'fuente_Direct Traffic',
 'ocupacion_Unemployed',
 'ocupacion_Working Professional',
 'paginas_vistas_mms',
 'score_actividad_mms',
 'score_perfil_mms',
 'tiempo_en_site_mms',
 'ult_actividad_Chat Conversation',
 'ult_actividad_Converted to Lead',
 'ult_actividad_Page Visited on Website',
 'ult_actividad_SMS Sent',
 'visitas_total_mms']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [7]:
variables_finales = ['ambito',
                   'fuente',
                   'ocupacion',
                   'paginas_vistas_visita',
                   'score_actividad',
                   'score_perfil',
                   'tiempo_en_site_total',
                   'ult_actividad',
                   'visitas_total'
                  ]

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

#### Por duplicados

In [8]:
df.drop_duplicates(inplace = True)

In [9]:
df.shape

(7521, 20)

#### Por EDA

In [10]:
df = df.loc [(df.ult_actividad != "Email Bounced") ]

In [11]:
df.shape

(7229, 20)

#### Para x

Quedarse solo con las de la lista.

In [12]:
df

,origen,fuente,no_enviar_email,no_llamar,compra,visitas_total,tiempo_en_site_total,paginas_vistas_visita,ult_actividad,ambito,ocupacion,conociste_google,conociste_revista,conociste_periodico,conociste_youtube,conociste_facebook,conociste_referencias,score_actividad,score_perfil,descarga_lm
id,,,,,,,,,,,,,,,,,,,,
660737,API,Chat,No,No,0,0.0,0,0.00,Page Visited on Website,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660728,API,Organic Search,No,No,0,5.0,674,2.50,Email Opened,Select,Unemployed,No,No,No,No,No,No,15.0,15.0,No
660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.00,Email Opened,Business Administration,Student,No,No,No,No,No,No,14.0,20.0,Yes
660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.00,Unreachable,Media and Advertising,Unemployed,No,No,No,No,No,No,13.0,17.0,No
660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.00,Converted to Lead,Select,Unemployed,No,No,No,No,No,No,15.0,18.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579564,Landing Page Submission,Direct Traffic,Yes,No,1,8.0,1845,2.67,Email Marked Spam,IT Projects Management,Unemployed,No,No,No,No,No,No,15.0,17.0,No
579546,Landing Page Submission,Direct Traffic,No,No,0,2.0,238,2.00,SMS Sent,Media and Advertising,Unemployed,No,No,No,No,No,No,14.0,19.0,Yes
579545,Landing Page Submission,Direct Traffic,Yes,No,0,2.0,199,2.00,SMS Sent,Business Administration,Unemployed,No,No,No,No,No,No,13.0,20.0,Yes


In [13]:
x = df[variables_finales].copy()

In [14]:
x

,ambito,fuente,ocupacion,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,
660737,Select,Chat,Unemployed,0.00,15.0,15.0,0,Page Visited on Website,0.0
660728,Select,Organic Search,Unemployed,2.50,15.0,15.0,674,Email Opened,5.0
660727,Business Administration,Direct Traffic,Student,2.00,14.0,20.0,1532,Email Opened,2.0
660719,Media and Advertising,Direct Traffic,Unemployed,1.00,13.0,17.0,305,Unreachable,1.0
660681,Select,Google,Unemployed,1.00,15.0,18.0,1428,Converted to Lead,2.0
...,...,...,...,...,...,...,...,...,...
579564,IT Projects Management,Direct Traffic,Unemployed,2.67,15.0,17.0,1845,Email Marked Spam,8.0
579546,Media and Advertising,Direct Traffic,Unemployed,2.00,14.0,19.0,238,SMS Sent,2.0
579545,Business Administration,Direct Traffic,Unemployed,2.00,13.0,20.0,199,SMS Sent,2.0


#### Para y

Especificar la target.

In [16]:
target = 'compra'

Crear el y.

In [17]:
y = df[target].copy()

In [18]:
y

id
660737    0
660728    0
660727    1
660719    0
660681    1
         ..
579564    1
579546    0
579545    0
579538    1
579533    1
Name: compra, Length: 7229, dtype: int64

## CREAR EL PIPELINE

#### Crear la función

In [19]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7229 entries, 660737 to 579533
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ambito                 6585 non-null   object 
 1   fuente                 7209 non-null   object 
 2   ocupacion              5409 non-null   object 
 3   paginas_vistas_visita  7119 non-null   float64
 4   score_actividad        4068 non-null   float64
 5   score_perfil           4068 non-null   float64
 6   tiempo_en_site_total   7229 non-null   int64  
 7   ult_actividad          7139 non-null   object 
 8   visitas_total          7119 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 564.8+ KB


In [20]:
def calidad_datos(df):
    temp = df.astype({'visitas_total': 'Int64'})      

    def imputar_moda (variable):   
        return (variable.fillna(variable.mode()[0]))   
    var_imputar_moda = ["ocupacion","ambito"]
    temp[var_imputar_moda] = temp[var_imputar_moda].apply(imputar_moda)

    var_imputar_valor = ["fuente",
                         "ult_actividad"
                        ]
    valor = "desconocido"
    temp[var_imputar_valor] = temp[var_imputar_valor].fillna(valor)
    
    var_imputar_mediana = ["paginas_vistas_visita",
                           "score_actividad",
                           "score_perfil",
                           "tiempo_en_site_total",
                           "visitas_total"             
                          ]
    def imputar_mediana(variable):
        if pd.api.types.is_integer_dtype(variable):
            return(variable.fillna(int(variable.median())))
        else:
            return(variable.fillna(variable.median()))
        
    temp[var_imputar_mediana] = temp[var_imputar_mediana].apply(imputar_mediana)

    def agrupar_cat_raras(variable, criterio = 0.02):
        #Calcula las frecuencias
        frecuencias = variable.value_counts(normalize=True)
        #Identifica las que están por debajo del criterio
        temp = [cada for cada in frecuencias.loc[frecuencias < criterio].index.values]
        #Las recodifica en 'OTROS'
        temp2 = np.where(variable.isin(temp),'OTROS',variable)
        #Devuelve el resultado
        return(temp2)
    
    var_agrupar_cat_raras = ["ocupacion","ambito","ocupacion","ambito"]

    for variable in var_agrupar_cat_raras:
        temp[variable] = agrupar_cat_raras(temp[variable],criterio = 0.02)

    temp["visitas_total"] = temp["visitas_total"].clip(0,50)

    return(temp)

In [21]:
calidad_datos(x)

,ambito,fuente,ocupacion,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,ult_actividad,visitas_total
id,,,,,,,,,
660737,Select,Chat,Unemployed,0.00,15.0,15.0,0,Page Visited on Website,0
660728,Select,Organic Search,Unemployed,2.50,15.0,15.0,674,Email Opened,5
660727,Business Administration,Direct Traffic,Student,2.00,14.0,20.0,1532,Email Opened,2
660719,Media and Advertising,Direct Traffic,Unemployed,1.00,13.0,17.0,305,Unreachable,1
660681,Select,Google,Unemployed,1.00,15.0,18.0,1428,Converted to Lead,2
...,...,...,...,...,...,...,...,...,...
579564,IT Projects Management,Direct Traffic,Unemployed,2.67,15.0,17.0,1845,Email Marked Spam,8
579546,Media and Advertising,Direct Traffic,Unemployed,2.00,14.0,19.0,238,SMS Sent,2
579545,Business Administration,Direct Traffic,Unemployed,2.00,13.0,20.0,199,SMS Sent,2


In [22]:
# Comprobar que no nos de ningun nulo
calidad_datos(x).isna().sum()

ambito                   0
fuente                   0
ocupacion                0
paginas_vistas_visita    0
score_actividad          0
score_perfil             0
tiempo_en_site_total     0
ult_actividad            0
visitas_total            0
dtype: int64

#### Convertirla en transformer

In [23]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [24]:
var_ohe = ['ambito','ocupacion',"fuente","ult_actividad"]
ohe = OneHotEncoder(sparse_output = False, handle_unknown='ignore')

var_mms = ["paginas_vistas_visita","score_actividad","score_perfil","tiempo_en_site_total","visitas_total"]
mms = MinMaxScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [25]:
ct = make_column_transformer(
    (ohe, var_ohe),
    (mms, var_mms),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [26]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [27]:
modelo = RandomForestRegressor(n_jobs = -1, 
                       max_depth = 5,
                       n_estimators = 50,
                       )

#### Crear el pipe final de entrenamiento

In [28]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [ ]:
# 1. Definir rutas de forma segura (mejor con os.path)
ruta_base = r'C:\Users\DataScience\ONLINE_DS_THEBRIDGE_DaniCastillo\ML_LEAD SCORING\01_src\01_data_sample\TRABAJO'
nombre_archivo = 'pipe_entrenamiento.pickle'
ruta_completa = os.path.join(ruta_base, nombre_archivo)

# 2. Asegurar que existe el directorio
os.makedirs(os.path.dirname(ruta_completa), exist_ok=True)

# 3. Guardar el pipeline
with open(ruta_completa, 'wb') as file:
    cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [31]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [32]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [33]:
ruta_base = r'C:\Users\DataScience\ONLINE_DS_THEBRIDGE_DaniCastillo\ML_LEAD SCORING\01_src\01_data_sample\TRABAJO'
ruta_completa = os.path.join(ruta_base, nombre_pipe_ejecucion)

# Guardar el modelo
with open(ruta_completa, 'wb') as file:
    cloudpickle.dump(pipe_ejecucion, file)